# Helpers

In [ ]:
def read(path):
    with open(path) as f:
        print(''.join(f.readlines()))

# Regression

Let's generate data of the following form: <br>
Every example has single namespace 'f' with single feature 'x' in it <br>
Target function is $$\hat{y} = 2x + 1$$
And we are learning weights $w$, $b$ for $$y=wx+b$$

In [ ]:
import numpy as np

with open('regression1.txt', 'w') as f:
    for i in range(1000):
        x = np.random.rand()
        y = 2 * x + 1
        f.write(f'{y} |f x:{x}\n')

## Simplest execution

In [ ]:
!vw -d regression1.txt

## Output more artifacts
-p - predictions <br>
--invert_hash - model in readable format <br>
-f - model in binary format (consumable by vw)

In [ ]:
!vw -d regression1.txt -p regression1.pred --invert_hash regression1.model.txt -f regression1.model.bin


We can look at weights and see the $w$ and $b$ got close to expected 2 and 1 values

In [ ]:
read('regression1.model.txt')

## Do only predictions, no learning

In [ ]:
!vw -d regression1.txt -t

In [ ]:
!vw -d regression1.txt -t --learning_rate 10

In [ ]:
!vw -d regression1.txt -t -i regression1.model.bin

## Interactions

Let's generate another dataset of the following form: <br>
Every example has single namespace 'f' with single feature 'x' in it <br>
Target function is $$\hat{y} = x^2 + 1$$

In [ ]:
import numpy as np

with open('regression2.txt', 'w') as f:
    for i in range(1000):
        x = np.random.rand() * 4
        y = x**2 + 1
        f.write(f'{y} |y x:{x} y:{np.random.random()}\n')

We can see loss being far from zero if we stil try to learn $$y=wx+b$$ 

In [ ]:
!vw -d regression2.txt --invert_hash regression2.model.txt

So let's try to learn $$y=w_1 x^2 + w_2 x + b$$

In [ ]:
!vw -d regression2.txt --invert_hash regression2.model.txt --interactions yy

In [ ]:
read('regression2.model.txt')

# Contextual bandits

In [ ]:
import numpy as np
env = {
    'Tom': {
        'sports': 0,
        'politics': 1
        },
    'Anna': {
        'sports': 1,
        'politics': 0
    }
}

users = ['Tom', 'Anna']
content = ['sports', 'politics']

with open('cb.txt', 'w') as f:
    for i in range(1000):
        user = users[np.random.randint(0, 2)]
        chosen = np.random.randint(0, 2)
        reward = env[user][content[chosen]]

        f.write(f'shared |u {user}\n')
        f.write(f'0:{-reward}:0.5 |a {content[chosen]}\n')
        f.write(f'|a {content[(chosen + 1) % 2]}\n\n')

Let's try to learn to predict reward in the following form: $$r = w_1 I(user\ is\ Tom) + w_2 I(user\ is\ Anna) + w_3 I(topic\ is\ sports) + w_4 I(topic\ is\ politics) + b$$

In [ ]:
!vw --cb_explore_adf -d cb.txt --invert_hash cb.model.txt

We can see that average reward is still around 0.5 which is the same as we would get answering randomly. This is expected since personalization is not captured in this form.
Let's add interaction between 'u' and 'a' namespaces and try to learn function of the following form:
$$\begin{aligned}r = w_1 I(user\ is\ Tom) I(topic\ is\ sports) + w_2 I(user\ is\ Tom) I(topic\ is\ politics) +\\+ w_3 I(user\ is\ Anna) I(topic\ is\ sports) + w_4 I(user\ is\ Anna) I(topic\ is\ politics) +\\+ w_5 I(user\ is\ Tom) + w_6 I(user\ is\ Anna) +\\+ w_7 I(topic\ is\ sports) + w_8 I(topic\ is\ politics) + b\end{aligned}$$

In [ ]:
!vw --cb_explore_adf -d cb.txt --invert_hash cb.model.txt --interactions ua

In [ ]:
read('cb.model.txt')